In [ ]:
from core.helpers.session_helper import SessionHelper
session = SessionHelper().session

import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
"""
************ CONFIGURATION - PLEASE TOUCH **************
Pipeline Builder configuration: creates configurations from variables specified here!!
This cell will be off in production as configurations will come from the configuration postgres DB.
"""
# config vars: this dataset
config_pharma = "bi" # the pharmaceutical company which owns {brand}
config_brand = "ofev" # the brand this pipeline operates on
config_state = "enrich" # the state this transform runs in
config_name = "accredo_cancel_before_active" # the name of this transform, which is the name of this notebook without .ipynb

# input vars: dataset to fetch. Recall that a contract published to S3 has a key format branch/pharma/brand/state/name
input_pharma = "bi"
input_brand = "ofev"
input_state = "ingest"
input_name = "patient_status_ingest_column_mapping"
input_branch = "dc-567_patient_status_ingest_mapping" # if None, input_branch is automagically set to your working branch

In [3]:
"""
************ SETUP - DON'T TOUCH **************
Populating config mocker based on config parameters...
"""
import core.helpers.pipeline_builder as builder

ids = builder.build(config_pharma, config_brand, config_state, config_name, session)
transform_id = ids[0]
run_id = ids[1]

2019-08-19 18:33:41,350 - core.logging - DEBUG - Adding/getting mocks for specified configurations...
2019-08-19 18:33:41,385 - core.logging - DEBUG - Done. Creating mock run event and committing results to configuration mocker.


In [4]:
"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)


# CORE Cartridge Notebook::accredo_cancel_before_active
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

### Configuration

In [5]:
""" 
********* VARIABLES - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<variable_name>: <data_type> #<comment explaining what the value is to future us>

e.g.

class Transform(DbTransform):
    some_ratio: float
    site_name: str

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    '''
    # Column headers
    status_date: str # Status date column
    ref_date: str # Referral date column
    patient: str # Patient ID column
    pharm: str # Pharmacy Name column
    product: str # Medication Name column
    status: str # Status column
    substatus: str # Substatus column
    ic_status: str # IntegriChain Status column
    ic_substatus: str # IntegriChain Substatus column
    pjh: str # Patient Journey Hierarchy column
    brand: str # Brand column (medication without strength)
        
    # Possible status values
    pending: str # Pending status 'PENDING'
    active: str # Active status 'ACTIVE'
    cancelled: str # Cancelled status 'CANCELLED'
    discontinued: str # Discontinued status 'DISCONTINUED'
        
    # Possible substatus values
    pending_new: str # New substatus when status is 'PENDING'
    active_shipped: str# Shipment substatus when status is 'ACTIVE'
    
    # Possible PJH values
    no_clarity: str # Final result of enrichment
        
    # Pharmacy this transform applies to
    accredo: str

transform = Transform()

In [6]:
## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull from the configuration application instead
## For the last example, this could look like...
## transform.some_ratio = 0.6
## transform.site_name = "WALGREENS"

# Vars
transform.patient = 'patient_pharmacy_id'
transform.pharm = 'pharmacy_code'
transform.product = 'medication'
transform.ref_date = 'referral_date'
transform.status_date = 'status_date'
transform.status = 'customer_status'
transform.substatus = 'customer_substatus'
transform.ic_status = 'integrichain_status'
transform.ic_substatus = 'integrichain_sub_status'
transform.pjh = 'Patient_Journey_Hierarchy'
transform.brand = 'brand'

# Values
transform.pending = 'PENDING'
transform.active = 'ACTIVE'
transform.cancelled = 'CANCELLED'
transform.discontinued = 'DISCONTINUED'
transform.pending_new = 'NEW'
transform.active_shipped = 'SHIPMENT'
transform.no_clarity = 'NO STATUS CLARITY'
transform.accredo = 'ACCREDO'

### Description

![what my transform does](assets/ds301_accredo_enrichment.png)

### Transformation

In [7]:
"""
************ FETCH DATA - TOUCH, BUT CAREFULLY **************
This cell will be turned off in production, as the input_contract will be handled by the pipeline.
"""

if not input_branch:
    input_branch = BRANCH_NAME
input_contract = DatasetContract(branch=input_branch, state=input_state, parent=input_pharma, child=input_brand, dataset=input_name)
run_filter = []
run_filter.append(dict(partition="__metadata_run_id", comparison="==", values=[3]))
# IF YOU HAVE PUBLISHED DATA MULTIPLE TIMES, uncomment the above line and change the int to the run_id to fetch.
# Otherwise, you will have duplicate values in your fetched dataset!
final_dataframe = input_contract.fetch(filters=run_filter)

import os

pd.options.display.max_columns=999

os.chdir('{}'.format(os.path.expanduser('~')))
status_config = pd.read_csv('bi_statuses.csv').rename(columns={
    'bi_status_code': transform.substatus
})

df = final_dataframe.copy()

df = (
    df
    .loc[:,
        [transform.ref_date,
         transform.patient,
         transform.pharm,
         transform.product,
         transform.status_date,
         transform.status,
         transform.substatus]
        ]
    .fillna('NONE')
    .assign(**{
        transform.brand: lambda x:(
            x[transform.product].apply(lambda x: x.split()[0].strip())
        ),
        transform.status: lambda x:(
            x[transform.status].str.upper()
        ),
        transform.substatus: lambda x:(
            x[transform.substatus].str.upper()
        )
    })
    .assign(**{
        transform.status_date: lambda x:(
            pd.to_datetime(
                x[transform.status_date].str[:8].astype(str),
                errors='coerce'
        )),
        transform.ref_date: lambda x:(
            pd.to_datetime(
                x[transform.ref_date].str[:8].astype(str),
                errors='coerce'
        )),
        'min_status_date': lambda x:(
            x.groupby([transform.patient, transform.pharm, transform.brand])[transform.status_date].transform(min)
        )
    })
    .fillna(value={transform.ref_date: 'min_status_date'})
    .drop(columns=['min_status_date'])
    .merge(status_config, how='left', on=transform.substatus)
    .drop_duplicates()
    .sort_values(
        [transform.patient, transform.pharm, transform.brand, transform.status_date]
    )
)

df.loc[:, transform.pjh] = np.nan

final_dataframe = df.copy()

2019-08-19 18:33:41,644 - core.dataset_contract.DatasetContract - INFO - Fetching dataframe from s3 location s3://ichain-dev/dc-567_patient_status_ingest_mapping/bi/ofev/ingest/patient_status_ingest_column_mapping.


/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:1404: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [8]:
# Preserve original data
df = final_dataframe.copy()

In [9]:
# Sort and reset index to join on later
df = (
    df
    .sort_values([
        transform.patient,
        transform.pharm,
        transform.brand,
        transform.status_date,
        transform.ref_date,
        transform.status
    ])
    .reset_index(drop=False)
)

In [10]:
# Create secondary df to find the first active shipment date for
# each patient journey
min_shipped_df = (
    df
    .loc[
        (df[transform.ic_status] == transform.active) &
        (df[transform.ic_substatus] == transform.active_shipped)
    ]
    .groupby([transform.patient, transform.pharm, transform.brand])
    [transform.status_date]
    .min()
    .reset_index()
    .rename(columns={transform.status_date: 'first_shipped_date'})
)

In [11]:
# Join to main df to get first active shipment date for every
# patient that has one
df = (
    df
    .merge(
        min_shipped_df,
        how='left'
    )
)

In [12]:
# Create a column that can be grouped on later to determine the
# first date where a desired care occurs
df = (
    df
    .assign(status_spree=(
    ~(
        # Check to make sure patient journey for row above is the same
        (df[transform.patient].eq(df[transform.patient].shift(1))) &
        (df[transform.pharm].eq(df[transform.pharm].shift(1))) &
        (df[transform.brand].eq(df[transform.brand].shift(1))) &
        # Check to make sure statuses in row above are the same
        (df[transform.ic_status].eq(df[transform.ic_status].shift(1))) &
        (df[transform.ic_substatus].eq(df[transform.ic_substatus].shift(1)))
    )
    ).cumsum())
)

In [13]:
# Create column denoting the min status date of each spree
df = (
    df
    .assign(min_status_date=lambda x:(
        x.groupby('status_spree')[transform.status_date].transform(min)
    ))
)

In [14]:
# Create bool column to denote row being right above active shipment row
df = (
    df
    .assign(above_shipment_step=lambda x:(
        # Check to make sure patient journey in row below is the same
        (x[transform.patient].eq(x[transform.patient].shift(-1))) &
        (x[transform.pharm].eq(x[transform.pharm].shift(-1))) &
        (x[transform.brand].eq(x[transform.brand].shift(-1))) &
        # Check to make sure status in row below is active/shipped
        (x[transform.ic_status].shift(-1) == transform.active) &
        (x[transform.ic_substatus].shift(-1) == transform.active_shipped)
    ))
)

# Pick out sprees that are above a shipment
above_shipment_series = df.groupby('status_spree').above_shipment_step.any()

# Assign True to those sprees
df = (
    df
    .assign(above_shipment=lambda x:(
        x.status_spree.isin(above_shipment_series[above_shipment_series].index)
    ))
)


In [15]:
# Denote how many days between min_status_date and first_shipped_date
df = (
    df
    .assign(day_diff=(df['min_status_date'] - df['first_shipped_date']))
)

In [16]:
df.loc[
    (df[patient] == accredo)
    # Negative for day diff, because we only want status sprees that
    # occur BEFORE the first active shipment
    (df.day_diff < np.timedelta64(-2, 'D')) &
    (df.above_shipment == True) &
    (
        (df[transform.ic_status].isin([transform.cancelled, transform.discontinued])) |
        (
            (df[transform.ic_status] == transform.pending) &
            (df[transform.ic_substatus] == transform.pending_new)
        )
    ),
    # Change pjh to 'NO STATUS CLARITY' for the records that pass
    transform.pjh
] = transform.no_clarity

In [17]:
# Format the final dataframe ready for publication
final_dataframe = (
    df
    # Preserve original index
    .set_index('index')
    .sort_index()
    # Drop calculation columns
    .drop(columns=[
        'first_shipped_date',
        'status_spree',
        'min_status_date',
        'above_shipment_step',
        'above_shipment',
        'day_diff'
    ])
)

### Publish

In [18]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
transform.publish_contract.publish(final_dataframe, run_id, session)
session.close()

2019-08-19 18:34:14,322 - core.dataset_contract.DatasetContract - INFO - Publishing dataframe to s3 location s3://ichain-dev/ds301_accredo_enrichment/bi/ofev/enrich/accredo_cancel_before_active with run ID 641481.
2019-08-19 18:34:14,414 - core.dataset_contract.DatasetContract - DEBUG - Publishing dataframe to Redshift Spectrum database ichain_core to schema.table                 data_core.bi_ofev_accredo_cancel_before_active...
2019-08-19 18:34:15,169 - s3parq.publish_redshift - INFO - Running query to create schema: CREATE EXTERNAL SCHEMA IF NOT EXISTS data_core                 FROM DATA CATALOG                 database 'ichain_core'                 iam_role 'arn:aws:iam::265991248033:role/mySpectrumRole';


/usr/local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


ProgrammingError: (psycopg2.ProgrammingError) permission denied for database ichain_core

[SQL: CREATE EXTERNAL SCHEMA IF NOT EXISTS data_core                 FROM DATA CATALOG                 database 'ichain_core'                 iam_role 'arn:aws:iam::265991248033:role/mySpectrumRole';]
(Background on this error at: http://sqlalche.me/e/f405)